In [77]:
# Import necessary libraries
import os
import re
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models import CoherenceModel

# Preprocessing

In [78]:
# Function to load data from folder path
def load_data(folder_path):
    articles = []
    
	# Iterate through all files in the folder path
    for file in os.listdir(folder_path):
        with open(os.path.join(folder_path, file), "r", encoding="latin") as f:
            content = f.read()
            articles.append(content)
    return articles

In [80]:
# Function to preprocess the loaded data
def preprocess_data(article_files):
    cleaned_articles = []
     # Iterate through all articles in the input list   

    for article_file in article_files:
        # Split individual articles
        articles = article_file.split('')
        for article in articles:
	        # Remove new line characters
                text = re.sub(r"\s+", " ", article)
                # Remove numbers and punctuation
                text = re.sub(r"[^a-zA-Z\s]", "", text)
                # Make all characters lowercase
                text = text.lower()
                # Tokenize text into words
                tokens = word_tokenize(text)
	        # Remove stop words (common words with little meaning such as "the" or "a")
                stop_words = set(stopwords.words("english"))
                custom_stopwords = {"mr", "said", "u"}  # Add any additional stopwords here
                stop_words = stop_words.union(custom_stopwords)
	        # Remove any remaining short words and lemmatize 
                tokens = [token for token in tokens if token not in stop_words]
                tokens = [token for token in tokens if len(token) > 1 or token in {"i", "a"}]
                lemmatizer = WordNetLemmatizer()
                tokens = [lemmatizer.lemmatize(token) for token in tokens]
                tokens = [token for token in tokens if token != "u"]
                cleaned_articles.append(tokens)
    return cleaned_articles

In [81]:
# Function to create a corpus and dictionary from the preprocessed data
def create_corpus_dictionary(cleaned_articles):
    dictionary = corpora.Dictionary(cleaned_articles)
    corpus = [dictionary.doc2bow(article) for article in cleaned_articles]
    return corpus, dictionary

In [82]:
# Run preprocessing functions
folder_path = "Articles/"
article_files = load_data(folder_path)
cleaned_articles = preprocess_data(article_files)
corpus, dictionary = create_corpus_dictionary(cleaned_articles)

 world europe in turkey us hand is seen in nearly every crisis by tim arango  words  january   nytimescom feed nytfeed english copyright  the new york times company all rights reserved istanbul urkish officials accused the united states of abetting a failed coup last summer when the russian ambassador to turkey was assassinated last month the turkish press said the united states was behind the attack and once again after a gunman walked into an istanbul nightclub early on new year day and killed dozens the progovernment news media pointed a finger at the united states merica chief suspectne headline blared after the attack on twitter a turkish lawmaker referring to the name of the nightclub wrote hoever the triggerman is reina attack is an act of cia period turkey has been confronted with a cascade of crises that seem to have only accelerated as the syrian civil war has spilled across the border but the events have not pushed turkey closer to its nato allies conversely they have drifte

# Topic Models

In [64]:
# Function to train an LDA model
def train_lda(corpus, dictionary, num_topics, passes):
    lda_model = models.LdaModel(corpus=corpus,
                                num_topics=num_topics,
                                id2word=dictionary,
                                passes=passes,
                                random_state=13)
    return lda_model

In [65]:
# Function to print and plot the topics from an LDA model
def plot_topics(lda_model):
    num_topics = lda_model.num_topics
    topic_words = lda_model.print_topics(num_words=10)
    
    for topic in range(num_topics):
        print(f"Topic {topic}:")
        print(topic_words[topic])

In [66]:
# Function to run the LDA training and save the results to a file
def run_and_save_lda(corpus, dictionary, num_topics, passes, model_number, output_file):
    lda_model = train_lda(corpus, dictionary, num_topics, passes)

    # save the 4 models to Models/ folder
    temp_file = 'Models/lda_model_' + str(model_number+1)
    lda_model.save(temp_file)

    # Save the topic words to output files
    topic_words = lda_model.print_topics(num_words=10)
    with open(output_file, "w") as f:
        for topic in range(num_topics):
            f.write(f"Topic {topic}:\n")
            f.write(str(topic_words[topic]) + "\n\n")

In [67]:
# Define different parameters to build the LDA model
# use different topic and pass count and discuss differences in report
params = [{"num_topics": 5, "passes": 10},{"num_topics": 10, "passes": 10},{"num_topics": 5, "passes": 20},{"num_topics": 10, "passes": 20}]


# Iterating through the parameters and building different LDA models
for i, p in enumerate(params):
    output_file = f"output_{i + 1}.txt"
    run_and_save_lda(corpus, dictionary,  p["num_topics"], p["passes"], i, output_file)

# Evaluation

In [68]:
# Load models from Models folder
model_1 = models.ldamodel.LdaModel.load("Models/lda_model_1")
model_2 = models.ldamodel.LdaModel.load("Models/lda_model_2")
model_3 = models.ldamodel.LdaModel.load("Models/lda_model_3")
model_4 = models.ldamodel.LdaModel.load("Models/lda_model_4")
models_list = [model_1, model_2, model_3, model_4]

In [69]:
# Function to print Coherence Score of model
def compute_coherence(lda_model):
    coherence_lda_model = CoherenceModel(model=lda_model, texts=cleaned_articles, dictionary=dictionary, coherence='c_v')
    coherence_number = coherence_lda_model.get_coherence()
    print('Coherence:  ', coherence_number)

In [70]:
model_number = 0

# Compute Perplexity of all 4 models
for m in models_list:
    model_number+=1
    print('Model ', model_number)
    print('Perplexity: ', m.log_perplexity(corpus))
    compute_coherence(m)
    print('\n')

Model  1
Perplexity:  -8.492769328374036
Coherence:   0.3008167104770959


Model  2
Perplexity:  -8.498756589638418
Coherence:   0.3068762045774803


Model  3
Perplexity:  -8.483742503821002
Coherence:   0.2884899369530641


Model  4
Perplexity:  -8.49175200324849
Coherence:   0.30918349721905114




# Visualization

In [71]:
# Enable Python Notebook display of pyLDAvis
pyLDAvis.enable_notebook()


In [72]:

# Model 1 visualization
vis = pyLDAvis.gensim_models.prepare(model_1, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.012922  0.017284       1        1  29.472214
3     -0.057745  0.021717       2        1  27.300731
4     -0.002677  0.007891       3        1  19.936411
1     -0.007610 -0.057973       4        1  15.975446
2      0.055111  0.011081       5        1   7.315198, topic_info=             Term         Freq        Total Category  logprob  loglift
2206         time  5574.000000  5574.000000  Default  30.0000  30.0000
1418          new  5911.000000  5911.000000  Default  29.0000  29.0000
2453         york  3557.000000  3557.000000  Default  28.0000  28.0000
40136       gigot  1097.000000  1097.000000  Default  27.0000  27.0000
2251        trump  7469.000000  7469.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2268          two   163.229238  2103.950897   Topic5  -6.1642   0.0588
1661    president   192.622362  4224.424739   Topic5  -5.9986  -0.4727
940    government   170.660070  2667.553384   Topic5  -6.1197  -0.1340
1482     official   167.113432  3317.380776   Topic5  -6.1407  -0.3730
2148        syria   164.504498  3956.474016   Topic5  -6.1564  -0.5650

[471 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
25047      3  0.880260     abballa
25047      4  0.055016     abballa
17089      1  0.970665       abedi
17089      2  0.005304       abedi
17089      3  0.015913       abedi
...      ...       ...         ...
33848      3  0.932138        zuck
12533      1  0.008140  zuckerberg
12533      2  0.081402  zuckerberg
12533      3  0.879143  zuckerberg
12533      4  0.024421  zuckerberg

[1045 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 2, 3])

In [73]:
# Model 2 visualization
vis = pyLDAvis.gensim_models.prepare(model_2, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.006923 -0.026163       1        1  18.903075
7      0.020930  0.036829       2        1  16.983255
9      0.060560  0.061449       3        1  13.154182
8      0.028415  0.017142       4        1  11.115286
4      0.009897 -0.027502       5        1   8.599334
6     -0.018413  0.010114       6        1   7.696896
1     -0.028926 -0.007541       7        1   6.946141
2      0.020579 -0.049475       8        1   6.279867
5      0.004916 -0.046427       9        1   5.578223
3     -0.104881  0.031573      10        1   4.743740, topic_info=             Term          Freq         Total Category  logprob  loglift
2251        trump   7440.000000   7440.000000  Default  30.0000  30.0000
2293       united   3293.000000   3293.000000  Default  29.0000  29.0000
2079        state  10018.000000  10018.000000  Default  28.0000  28.0000
1418          new   5969.000000   5969.000000  Default  27.0000  27.0000
40136       gigot   1068.000000   1068.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
867         force    122.916384   3508.432118  Topic10  -6.0147  -0.3031
2432        would    122.637472   3758.466641  Topic10  -6.0170  -0.3742
940    government    116.553833   2653.423160  Topic10  -6.0679  -0.0769
1339     military    116.813870   3244.602032  Topic10  -6.0657  -0.2758
1115          isi    111.204537   2730.149509  Topic10  -6.1149  -0.1524

[858 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0          1  0.109153       abadi
0          2  0.496644       abadi
0          3  0.087322       abadi
0          4  0.125525       abadi
0          5  0.076407       abadi
...      ...       ...         ...
12533      3  0.015729  zuckerberg
12533      4  0.865103  zuckerberg
12533      6  0.023594  zuckerberg
12533      9  0.007865  zuckerberg
35866      8  0.828075        zway

[2509 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 10, 9, 5, 7, 2, 3, 6, 4])

In [74]:
# Model 3 visualization
vis = pyLDAvis.gensim_models.prepare(model_3, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.010598  0.011560       1        1  30.530995
3     -0.062816  0.030866       2        1  27.714175
4     -0.005020 -0.000210       3        1  18.994693
1     -0.008237 -0.062125       4        1  16.345430
2      0.065475  0.019909       5        1   6.414707, topic_info=            Term         Freq        Total Category  logprob  loglift
1418         new  5962.000000  5962.000000  Default  30.0000  30.0000
2206        time  5635.000000  5635.000000  Default  29.0000  29.0000
40136      gigot  1048.000000  1048.000000  Default  28.0000  28.0000
2453        york  3610.000000  3610.000000  Default  27.0000  27.0000
2251       trump  7436.000000  7436.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1661   president   164.961467  4212.241177   Topic5  -6.0223  -0.4935
520      country   152.339290  3121.807244   Topic5  -6.1019  -0.2735
1482    official   142.731766  3293.140095   Topic5  -6.1670  -0.3921
2148       syria   142.077490  3936.665088   Topic5  -6.1716  -0.5751
1928    security   138.355103  2046.660760   Topic5  -6.1982   0.0524

[459 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
25047      3  0.902795     abballa
25047      4  0.053106     abballa
17089      1  0.992387       abedi
40023      1  0.021810    abihabib
40023      2  0.981462    abihabib
...      ...       ...         ...
33848      3  0.898884        zuck
12533      1  0.007894  zuckerberg
12533      2  0.078936  zuckerberg
12533      3  0.891980  zuckerberg
12533      4  0.023681  zuckerberg

[939 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 2, 3])

In [75]:
# Model 4 visualization
vis = pyLDAvis.gensim_models.prepare(model_4, corpus, dictionary)
vis

c:\Users\Dustin\anaconda3\envs\textminingclass\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.009969 -0.021651       1        1  19.545135
7      0.029294  0.037881       2        1  16.109860
9      0.058083  0.068527       3        1  14.554601
8      0.025822  0.013564       4        1  10.511218
4      0.011946 -0.030963       5        1   8.068432
6     -0.022444  0.008108       6        1   7.605629
1     -0.026850 -0.002564       7        1   6.943471
2      0.021281 -0.042425       8        1   6.279603
5      0.010192 -0.058148       9        1   5.396839
3     -0.117294  0.027672      10        1   4.985211, topic_info=             Term          Freq         Total Category  logprob  loglift
2251        trump   7422.000000   7422.000000  Default  30.0000  30.0000
40136       gigot   1072.000000   1072.000000  Default  29.0000  29.0000
2293       united   3276.000000   3276.000000  Default  28.0000  28.0000
2079        state  10008.000000  10008.000000  Default  27.0000  27.0000
1418          new   5956.000000   5956.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
2432        would    129.446669   3763.272550  Topic10  -6.0126  -0.3711
1339     military    126.610680   3246.127708  Topic10  -6.0348  -0.2454
867         force    127.505943   3507.485590  Topic10  -6.0277  -0.3158
940    government    122.887997   2650.554504  Topic10  -6.0646  -0.0726
1867        right    124.096491   3239.863760  Topic10  -6.0548  -0.2635

[868 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0          1  0.109206       abadi
0          2  0.507808       abadi
0          3  0.098285       abadi
0          4  0.114666       abadi
0          5  0.070984       abadi
...      ...       ...         ...
12533      3  0.015820  zuckerberg
12533      4  0.877986  zuckerberg
12533      6  0.023729  zuckerberg
12533      9  0.007910  zuckerberg
35866      8  0.947741        zway

[2458 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 10, 9, 5, 7, 2, 3, 6, 4])